<h1>Hello and welcome to the Brain, Cognition, and Technology physiology analysis notebook!<h1>

<h3> Replace the example file name below with the name of the file you want to analyze (between the quotation marks). Then click on the green "RUN THE ANALYSIS" button. <h3>

In [252]:
File_name = "david_testing_2022-01-27_18-34-25.txt"

In [253]:
from IPython.display import Javascript, display
from ipywidgets import widgets
from ipywidgets import Button, Layout

def run_all(ev):
    display(Javascript('IPython.notebook.execute_all_cells()'))

button = widgets.Button(description="RUN THE ANALYSIS",layout=Layout(width='50%', height='80px'))
button.style.button_color = 'lightgreen'
button.on_click(run_all)
display(button)

<IPython.core.display.Javascript object>

Button(description='RUN THE ANALYSIS', layout=Layout(height='80px', width='50%'), style=ButtonStyle(button_col…

In [254]:
import biosignalsnotebooks as bsnb
import numpy as np
import neurokit2 as nk
import pandas as pd 

# Scientific packages
from numpy import linspace, max, min, average, std, array, diff, fabs, sqrt, power, round
from scipy.integrate import simps

# Scientific programming package.
from numpy import average, array, reshape, sqrt, sort, diff, where, argmax, max
from numbers import Number

# Pacakge dedicated to Wavelet decomposition algorithms.
from pywt import swt, iswt

# Machine-learning dedicated package.
from sklearn.mixture import GaussianMixture

# Gaussian Distribution function.
from scipy.stats import norm

# Built-in packages.
from copy import deepcopy

In [255]:
Data_file = "/Users/a1/Downloads/" + File_name

<h2> Respiration (Breathing) <h2>

In [256]:
#Opensignals Device setting
ch = 'CH1'
sr = 1000
resolution = 16 # Resolution (number of available bits)

In [257]:
#DATA LOADOUT 
data, header = bsnb.load(Data_file,get_header=True)

In [258]:
signal = data[ch]
#convert signal according to bsnb guidelines
signal_converted = ((np.array(signal) / 2**resolution) - 0.5) * 100
#filtering signal for bandwidth of 0.1- 0.35 Hz, which is 6 Breathings to 21 Breathings per minute
filtered_signal = nk.rsp_clean(signal_converted, sampling_rate = sr, method = "biosppy")
process, info = nk.rsp_process(filtered_signal, sampling_rate= sr)
analysis = nk.rsp_analyze(process)

analysis

,RSP_Rate_Mean,RSP_Amplitude_Mean,RSP_Phase_Duration_Inspiration,RSP_Phase_Duration_Expiration,RSP_Phase_Duration_Ratio,RRV_RMSSD,RRV_MeanBB,RRV_SDBB,RRV_SDSD,RRV_CVBB,...,RRV_DFA_alpha1,RRV_DFA_alpha1_ExpRange,RRV_DFA_alpha1_ExpMean,RRV_DFA_alpha1_DimRange,RRV_DFA_alpha1_DimMean,RRV_DFA_alpha2,RRV_DFA_alpha2_ExpRange,RRV_DFA_alpha2_ExpMean,RRV_DFA_alpha2_DimRange,RRV_DFA_alpha2_DimMean
0,14.787968,22.098771,2.048572,2.04971,0.999445,1423.228671,4101.64,1057.506425,1426.79516,0.257825,...,0.693406,0.983506,0.890855,0.759486,0.693125,0.608975,0.682907,0.77284,0.708476,0.675007


<h2> ECG (Heart Rate Variability) <h2>

In [259]:
#Opensignals Device setting
channel = 'CH2'
data, header = bsnb.load("/Users/a1/Downloads/david_testing_2022-01-27_18-34-25.txt", get_header=True)

In [260]:
# Sampling frequency and acquired data
fs = header["sampling rate"]

# Signal Samples
signal = data[channel]
time = linspace(0, len(signal) / fs, len(signal))
tachogram_data, tachogram_time = bsnb.tachogram(signal, fs, signal=True, out_seconds=True)

In [261]:
tachogram_data_NN, tachogram_time_NN = bsnb.remove_ectopy(tachogram_data, tachogram_time)
bpm_data = (1 / array(tachogram_data_NN)) * 60

In [262]:
# Maximum, Minimum and Average RR Interval
max_rr = max(tachogram_data_NN)
min_rr = min(tachogram_data_NN)
avg_rr = average(tachogram_data_NN)

# Maximum, Minimum and Average Heart Rate
max_hr = 1 / min_rr # Cycles per second
max_bpm = max_hr * 60 # BPM

min_hr = 1 / max_rr # Cycles per second
min_bpm = min_hr * 60 # BPM

avg_hr = 1 / avg_rr # Cyles per second
avg_bpm = avg_hr * 60 # BPM

# SDNN
sdnn = std(tachogram_data_NN)

time_param_dict = {"Maximum RR": max_rr, "Minimum RR": min_rr, "Average RR": avg_rr, "Maximum BPM": max_bpm, "Minimum BPM": min_bpm, "Average BPM": avg_bpm, "SDNN": sdnn}

In [263]:
# Auxiliary Structures
tachogram_diff = diff(tachogram_data)
tachogram_diff_abs = fabs(tachogram_diff)
sdsd = std(tachogram_diff)
rr_i = tachogram_data[:-1]
rr_i_plus_1 = tachogram_data[1:]

# PoincarÃ© Parameters
sd1 = sqrt(0.5 * power(sdsd, 2))
sd2 = sqrt(2 * power(sdnn, 2) - power(sd1, 2))
sd1_sd2 = sd1 / sd2

In [264]:
# Auxiliary Structures
freqs, power_spect = bsnb.psd(tachogram_time, tachogram_data) # Power spectrum.

# Frequemcy Parameters
freq_bands = {"ulf_band": [0.00, 0.003], "vlf_band": [0.003, 0.04], "lf_band": [0.04, 0.15], "hf_band": [0.15, 0.40]}
power_values = {}
total_power = 0

band_keys = freq_bands.keys()
for band in band_keys:
    freq_band = freq_bands[band]
    freq_samples_inside_band = [freq for freq in freqs if freq >= freq_band[0] and freq <= freq_band[1]]
    power_samples_inside_band = [p for p, freq in zip(power_spect, freqs) if freq >= freq_band[0] and freq <= freq_band[1]]
    power = round(simps(power_samples_inside_band, freq_samples_inside_band), 5)
    
    # Storage of power inside each band
    power_values[band] = power
    
    # Total power update
    total_power = total_power + power

In [265]:
# Number of RR intervals that have a difference in duration, from the previous one, of at least 20 ms
nn20 = sum(1 for i in tachogram_diff_abs if i > 0.02)
pnn20 = int(float(nn20) / len(tachogram_diff_abs) * 100) # Percentage value.

# Number of RR intervals that have a difference in duration, from the previous one, of at least 50 ms
nn50 = sum(1 for i in tachogram_diff_abs if i > 0.05)
pnn50 = int(float(nn50) / len(tachogram_diff_abs) * 100) # Percentage value.

In [266]:
dictParameters = bsnb.hrv_parameters(signal, fs, signal=True)
print (dictParameters)

{'MaxRR': 1.0570012692138278, 'MinRR': 0.6880008261297235, 'AvgRR': 0.9178429729409757, 'MaxBPM': 87.20919760739781, 'MinBPM': 56.764359464418206, 'AvgBPM': 65.37065900035871, 'SDNN': 0.0582618586344522, 'SD1': 0.05834178906954914, 'SD2': 0.05818181839067958, 'SD1/SD2': 1.0027494960332004, 'NN20': 462, 'pNN20': 51, 'NN50': 69, 'pNN50': 7, 'ULF_Power': 0.0, 'VLF_Power': 0.00244, 'LF_Power': 0.0013, 'HF_Power': 0.00216, 'LF_HF_Ratio': 0.6018518518518519, 'Total_Power': 0.0059}


<h2> EDA (Galvanic Skin Response) <h2>

In [267]:
#Opensignals Device setting
data, header = bsnb.load("/Users/a1/Downloads/david_testing_2022-01-27_18-34-25.txt", get_header=True)
signal = data["CH3"]

In [268]:
# Sampling rate definition.
sr = header["sampling rate"]

# Raw to uS sample value conversion.
signal_us = bsnb.raw_to_phy("EDA", "bitalino_rev", signal, 10, "uS")

In [269]:
# Conventional 1st order Butterworth bandpass filtering 
# (due to the restrictive bandwidth in order to guarantee system stability it is necessary to reduce the filter order)
# [Option RES]
signal_res = bsnb.bandpass(signal_us, 0.045, 0.25, order=1, fs=sr, use_filtfilt=True)

# Conventional 2nd order Butterworth lowpass filtering 
# [Option EXT]
signal_ext = bsnb.lowpass(signal_us, 35, order=2, fs=sr, use_filtfilt=True)

In [270]:
# SWT 8th level Decomposition using "Haar" mother wavelet (taking into consideration its ability to detect "edges" in the signal) .
swt_orig_coeffs = swt(signal_ext[:32768], "haar", level=8)

# Restriction of filtering algorithm to the coefficients of the last decomposition level.
detail_coeffs = swt_orig_coeffs[0][1]
scaling_coeffs = swt_orig_coeffs[0][0]

In [271]:
gaussian_mixt = GaussianMixture(n_components=2, covariance_type="spherical") # "spherical" covariance type ensures that each Gaussian components has its own single variance.

# Reshape data to a column vector format.
detail_coeffs_col = reshape(detail_coeffs, (len(detail_coeffs), 1))

# Fit data to our model object.
gaussian_mixt.fit(detail_coeffs_col)

GaussianMixture(covariance_type='spherical', n_components=2)

In [272]:
# Normal distribution function objects.
norm_1 = norm(loc=gaussian_mixt.means_[0][0], scale=sqrt(gaussian_mixt.covariances_[0])) 
norm_2 = norm(loc=gaussian_mixt.means_[1][0], scale=sqrt(gaussian_mixt.covariances_[1])) 

# Component weights.
weight_1 = gaussian_mixt.weights_[0]
weight_2 = gaussian_mixt.weights_[1]

# CDF values for the coefficients under analysis.
sort_detail_coeffs = sort(detail_coeffs)
norm_1_cdf = norm_1.cdf(sort_detail_coeffs)
norm_2_cdf = norm_2.cdf(sort_detail_coeffs)

# CDF of the Gaussian mixture.
cdf_mixt = weight_1 * norm_1_cdf + weight_2 * norm_2_cdf

In [273]:
art_prop = 0.01 # Artifact proportion value.
low_thr = None 
high_thr = None

# Check when the CDF mixture function reaches values art_prop / 2 and 1 - art_prop / 2.
for i in range(0, len(norm_1_cdf)):
    # Low threshold clause.
    if cdf_mixt[i] - cdf_mixt[0] >= art_prop and low_thr == None:
        low_thr = sort_detail_coeffs[i]
        
    # High threshold clause.
    if cdf_mixt[-1] - cdf_mixt[i] <= art_prop and high_thr == None:
        high_thr = sort_detail_coeffs[i]

In [274]:
filt_detail_coeffs = deepcopy(detail_coeffs)
count_1 = 0
count_2 = 0
for j in range(0, len(filt_detail_coeffs)):
    if detail_coeffs[j] <= low_thr or detail_coeffs[j] >= high_thr:
        filt_detail_coeffs[j] = 0
    else:
        continue
        
# Update of the SWT decomposition tupple.
swt_coeffs = [(array(scaling_coeffs), array(filt_detail_coeffs))]

In [275]:
rec_signal = iswt(swt_coeffs, "haar")

In [276]:
signal_int = bsnb.smooth(rec_signal, sr * 3) # Moving average witt window size equal to 3 x sampling rate.

# Rescale signal.
signal_int = signal_int * (max(signal_ext) / max(signal_int))

In [277]:
# Parameter extraction.
param_dict = {}

#[Latency to stimulus onset]
signal_2nd_der = diff(diff(signal_int)) # Generation of 2nd derivative function.
der_thr = max(signal_2nd_der) # Identification of our major concavity point (start of response time)
response_sample = argmax(signal_2nd_der)
response_time = response_sample / sr
param_dict["Latency to stimulus onset"] = response_time - 0

#[EDR amplitude]
eda_max = max(signal_int)
eda_basal = signal_int[response_sample]
param_dict["EDR amplitude"] = eda_max - eda_basal

#[EDR rising time (Rise Time)]
eda_max_sample = argmax(signal_int)
eda_max_time = eda_max_sample / sr
param_dict["EDR rising time (Rise Time)"] = eda_max_time - response_time

#[EDR response peak (Peak Time)]
param_dict["EDR response peak (Peak Time)"] = eda_max_time

#[Recovery time to 50% amplitude]
time_50 = None
for i in range(eda_max_sample, len(signal_int)):
    if signal_int[i] <= eda_max - 0.50 * param_dict["EDR amplitude"]:
        time_50 = i / sr
        break
param_dict["Recovery time to 50% amplitude"] = time_50 - eda_max_time

#[Recovery time to 63% amplitude]
time_63 = None
for i in range(eda_max_sample, len(signal_int)):
    if signal_int[i] <= eda_max - 0.63 * param_dict["EDR amplitude"]:
        time_63 = i / sr
        break
param_dict["Recovery time to 63% amplitude"] = time_63 - eda_max_time

In [278]:
param_dict

{'Latency to stimulus onset': 32.024,
 'EDR amplitude': 6.6245598475980785,
 'EDR rising time (Rise Time)': -32.024,
 'EDR response peak (Peak Time)': 0.0,
 'Recovery time to 50% amplitude': 14.09,
 'Recovery time to 63% amplitude': 16.618}